<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/translate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

想把 HuggingFaceTB/cosmopedia 英文数据中的prompt和text 翻译成 中文， 然后看了下 deep_translator的实现， 翻译调用的是三方接口集成库， 于是使用这个库封装的谷歌翻译接口来翻译，但是三方平台接口多会有限流和接口调用频率限制，即使在代码中有容错处理， 比如常规的sleep下再调用，不影响整理处理流程，但是整体处理时间受接口限制，即使用批处理也如此，这个在大规模数据处理中使用三方接口时，经常会遇到的问题，用的三方服务，在技术上不好解决； 于是选择另外一种方案，看是否有开源的翻译模型，底层模型结构一般也是Transform架构 Encoder-decoder model ，也称sequence-to-sequence model； 比如 谷歌的T5模型， 但是推理速度受硬件条件影响，比较慢， 然后看了下meta nllb 模型，专门用来处理翻译的模型，单个 AI 模型中支持 200 种语言，开源地址： https://github.com/facebookresearch/fairseq/tree/nllb  

模型相对现在的LLM参数量小一些，也在huggingface的Transforms库中集成 nllb-200-distilled-600M，直接可以load使用， 等等。。。 既然llm推理可以通过想llama.cpp通过加载ggml格式进行量化，在性能有少许折损的情况下降低推理成本，但是ggml格式还不支持nllb模型权重文件；那就直接用Transforms库来加载facebook/nllb-200-distilled-600M 模型来批量翻译试试看

- 使用Transforms库进行翻译任务： https://huggingface.co/docs/transformers/tasks/translation

- https://github.com/facebookresearch/fairseq/tree/nllb

- https://ai.meta.com/blog/nllb-200-high-quality-machine-translation/


In [11]:
!pip install -q -U datasets deep-translator transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 14.9 MB/s eta 0:00:00


In [7]:
!huggingface-cli download \
  --repo-type dataset HuggingFaceTB/cosmopedia data/stories/train-00000-of-00043.parquet \
  --local-dir ./datas/datasets/HuggingFaceTB/cosmopedia \
  --local-dir-use-symlinks False

Consider using `hf_transfer` for faster downloads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.
downloading https://huggingface.co/datasets/HuggingFaceTB/cosmopedia/resolve/main/data/stories/train-00000-of-00043.parquet to /root/.cache/huggingface/hub/tmpgfppksba
train-00000-of-00043.parquet: 100% 277M/277M [00:02<00:00, 105MB/s]
./datas/datasets/HuggingFaceTB/cosmopedia/data/stories/train-00000-of-00043.parquet


In [26]:
!cat translate.py

# https://huggingface.co/learn/nlp-course/chapter5/
# https://zhuanlan.zhihu.com/p/564816807

from deep_translator import GoogleTranslator
from datasets import load_dataset, load_from_disk
import argparse
import time


def translate_en2cn(item):
    translated = ""
    try_cn = 100
    while try_cn > 0:
        try:
            translated_prompt = GoogleTranslator(source='en', target='zh-CN').translate(
                item["prompt"])
            translated_text = GoogleTranslator(source='en', target='zh-CN').translate(
                item["text"])
            break
        except Exception as e:
            print("An error occurred:", e)
            time.sleep(1)
            try_cn -= 1

    return {"text_zh": translated_text, "prompt_zh": translated_prompt}


def batch_check_data(batch):
    return [len(item) != 0 for item in batch["text_zh"]]


def translate2save(src_dataset_dir: str, target_dataset_dir: str):
    data = load_dataset(src_dataset_dir, split="train[:100]")
    data =

In [25]:
!python translate.py translate -s ./datas/datasets/HuggingFaceTB/cosmopedia/data/stories \
  -t ./datas/datasets/HuggingFaceTB/cosmopedia_zh

Map: 100% 100/100 [03:34<00:00,  2.15s/ examples]
Saving the dataset (1/1 shards): 100% 100/100 [00:00<00:00, 9226.77 examples/s]
Dataset({
    features: ['text', 'prompt', 'text_token_length', 'seed_data', 'format', 'audience', 'text_zh', 'prompt_zh'],
    num_rows: 100
})


In [18]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M", use_auth_token=True, src_lang="eng_Latn",target_lang="zho_Hans")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M", use_auth_token=True)



In [24]:
article = """
Dr. Mitchell nodded approvingly. "Yes, indeed! Fatty fish boast rich stores of omega-3 fatty acids – specifically EPA and DHA – which work miracles within our bodies. Amongst numerous benefits, they decrease triglyceride levels, lower blood pressure, discourage plaque formation, and even suppress irregular heartbeats."

An intriguing idea took root in Alex's mind, blossoming rapidly into resolve. If these simple changes could make such profound differences in his life, why stop there? Why not share this gift with others – friends, colleagues, strangers who might also benefit from understanding how food choices impact heart health? Perhaps he could host educational events, cooking demonstrations, or write articles championing these powerful culinary tools. Yes, he decided, that would be his mission – to spread awareness and inspire change, one plate at a time.

His thoughts must have shown on his face because Dr. Mitchell suddenly grinned, leaning back in her chair with satisfaction. "Ah, I see the wheels turning in that brilliant mind of yours, Alex. Go ahead, run with it!"

Encouraged, he dove deeper into conversation, eager to learn more about the remaining items on Dr. Mitchell's list: dark chocolate, tomatoes, avocados, legumes, and garlic & onions. Each ingredient held untapped potential, waiting to transform ordinary meals into extraordinary acts of self-care. By incorporating them regularly, Alex understood he wasn't merely eating for pleasure or sustenance – he was actively fighting for his future, nurturing himself and his loved ones toward longevity and happiness.

As they parted ways outside the café, promising to reconvene soon to discuss strategies for implementing these dietary shifts, both individuals felt invigorated by possibility. In sharing her wisdom, Dr. Mitchell had kindled a fire within Alex, illuminating pathways previously obscured by darkness and doubt. Armed with knowledge, fortified by friendship, and guided by love, they stepped boldly forward into a world ripe with opportunity – where every meal became an occasion for celebration, gratitude, and healing. --> Text length need to be between 0 and 5000 characters
An error occurred:  The midday sun streamed through the windows of the cozy café, casting dappled shadows onto the wooden tables and chairs. Dr. Sarah Mitchell, a renowned cardiologist, sat across from her longtime friend and patient, Alex. They had known each other since medical school, and their bond ran deep - woven together by years of shared experiences, laughter, and tears.

Alex fidgeted nervously with his coffee cup, staring intently into its swirling depths before finally breaking the silence. "Sarah," he began hesitantly, "you know I trust your judgment implicitly. You've saved my life countless times, and I am eternally grateful." He paused, taking a deep breath before continuing. "But this time feels different. This isn't just about me anymore; it's about my family too."
"""
inputs = tokenizer(article, return_tensors="pt")

translated_tokens = model.generate(**inputs, forced_bos_token_id=tokenizer.lang_code_to_id["zho_Hans"], max_length=1024)
tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]

'密切尔博士赞成. "是的,确实!脂肪鱼在他的生活中可以产生如此深刻的差异,为什么不停在那里? 特别是 EPA 和 DHA 在我们的身体中表现出奇迹.其中许多好处,它们降低三糖水平,降低血压,阻止片形成,甚至抑制心跳. "一个有趣的想法扎根在亚历克斯的脑海中,迅速勃发展. 如果这些简单的变化可以在他的生活中产生如此深刻的差异,为什么不停留在那里? 谢谢分享这个礼物与其他人. 朋友,同事,陌生人也可能从如何选择食物来打败我们的身体? 也许他可以主办教育活动,写演示,或通过这些强大的深草的工具来引导米切尔教授的椅子. 他决定,他的咖啡碗必须在一个时间内发育,并迅速激发自己的生活,因为他知道,在每一个时间内,他都会不断地分享他的知识,因为他知道,在每一个时间内,他都会在一个深深深的时间内,他和他的朋友们都会在一个深的时间内,因为他知道,他会在一个深深的时间内,他会在一个深的时间内,他会在一个深的时间里,他会在一个深的时间里,他知道,他会在一个深的时间里,他会在一个深的时间里,他会在一个深的时间里,他会发现,他会在一个深的时间里,他会在一个深的时间里,他有着他的心里,他有着自己的想法,他会在一个深的想法,他会在一个深的想法,他会在他的心里,他知道,他会在一个深的想法,他会在他的心里,他有着他的想法,他会在一个深的想法,他有着他的想法,他有着他的想法,他在他的想法,他在他的想法,他在他的想法,他在他的想法,他知道,他在他的想法,他在他的想法,在他的想法,在他的想法,在他的想法,在他的想法,在他的想法,在他的想法,在他的想法,在他的想法,在他的想法,在他的想法,在他的想法,在他的想法,在他的想法,在他的想法,在他的想法,在他的想法,在他的想法,在他的想法,在他的想法,在他的想法,在他的想法,在他的想法,在他的想法,在他的想法,在他的想法,在他的想法,在他的想法,在他的想法,在他的想法,在他的想法,在他的想法,在他的想法,在他的想法,在他的想法,在他的想法,在他的想法,在他的想法,在他的想法,在他的想法,在他的想法,在他的想法,在他的想法,在他的想法,在他的想法,在他的想法,在在在在他的想法,在在在在在在在在在他的想法,在在在在在在在在他的想法,在他的中.'